In [79]:
# Dependencies
import json
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime
from api_keys import DarkSkyKey

In [80]:
# read taxi rides file
file = "Resources/Taxi_Trips_Random_Sample.csv"
df = pd.read_csv(file, header=None)
df.columns = ["Trip ID", "Taxi ID","Trip Start Timestamp", "Trip End Timestamp","Trip Seconds",
             "Trip Miles", "Pickup Census Tract", "Dropoff Census Tract", "Pickup Community Area", 
             "Dropoff Community Area", "Fare", "Tips", "Tolls", "Extras", "Trip Total", "Payment Type", 
             "Company", "Pickup Centroid Latitude", "Pickup Centroid Longitude", "Pickup Centroid Location", 
             "Dropoff Centroid Latitude", "Dropoff Centroid Longitude", "Dropoff Centroid Location","Community Areas"]
df.head()

Trip ID  \
0  bc6f08423e9f194cb63a0d5ec6c6a2dea001e914   
1  b2e0ede2b0d907551beda4e8b4b83c143010f63d   
2  c2155aedc41a27673cd498d0fea754a18132d7f9   
3  c1e860354f05150021e1bdeb7696314ca3e15608   
4  bdff93a935333338e6f2f04f778ad7713a36c9ad   

                                             Taxi ID    Trip Start Timestamp  \
0  07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...  11/16/2013 01:45:00 AM   
1  74605d6aa0c8ba08190a5824f79ef27172c59671b00582...  09/28/2016 04:45:00 PM   
2  4243ab34318764eb5ca51054af503c610b50579594cef6...  12/10/2013 07:15:00 AM   
3  65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...  02/27/2014 09:00:00 AM   
4  8ca267f2246008a052c14ce8c81001149d86916ab274d3...  10/31/2016 04:15:00 PM   

       Trip End Timestamp  Trip Seconds  Trip Miles  Pickup Census Tract  \
0  11/16/2013 01:45:00 AM         480.0        0.15         1.703108e+10   
1  09/28/2016 04:45:00 PM         420.0        0.00         1.703184e+10   
2  12/10/2013 07:30:00 AM         720.0        1.80         1.703128e+10   
3  02/27/2014 09:45:00 AM        2700.0        0.00         1.703108e+10   
4  10/31/2016 04:15:00 PM         660.0        3.00                  NaN   

   Dropoff Census Tract  Pickup Community Area  Dropoff Community Area  \
0          1.703108e+10                    8.0                     8.0   
1          1.703132e+10                   32.0                    32.0   
2          1.703108e+10                   28.0                     8.0   
3          1.703198e+10                    8.0                    76.0   
4                   NaN                   32.0                    35.0   

        ...        Trip Total  Payment Type  \
0       ...              8.05          Cash   
1       ...              8.75   Credit Card   
2       ...              8.44          Cash   
3       ...             50.70   Credit Card   
4       ...             13.50   Credit Card   

                             Company  Pickup Centroid Latitude  \
0                                NaN                 41.907413   
1  Blue Ribbon Taxi Association Inc.                 41.880994   
2                                NaN                 41.879255   
3            Choice Taxi Association                 41.902788   
4                                NaN                 41.878866   

   Pickup Centroid Longitude              Pickup Centroid Location  \
0                 -87.640902  POINT (-87.6409015248 41.9074128162)   
1                 -87.632746  POINT (-87.6327464887 41.8809944707)   
2                 -87.642649   POINT (-87.642648998 41.8792550844)   
3                 -87.626146  POINT (-87.6261455896 41.9027880476)   
4                 -87.625192  POINT (-87.6251921424 41.8788655841)   

  Dropoff Centroid Latitude  Dropoff Centroid Longitude  \
0                 41.892042                  -87.631864   
1                 41.884987                  -87.620993   
2                 41.895033                  -87.619711   
3                 41.979071                  -87.903040   
4                 41.835118                  -87.618678   

              Dropoff Centroid Location Community Areas  
0  POINT (-87.6318639497 41.8920421365)            37.0  
1  POINT (-87.6209929134 41.8849871918)            38.0  
2  POINT (-87.6197106717 41.8950334495)            29.0  
3  POINT (-87.9030396611 41.9790708201)            37.0  
4  POINT (-87.6186777673 41.8351179863)            38.0  

[5 rows x 24 columns]

In [81]:
# convert timestamp format for api calls
newdatestart_list = []

for date in df['Trip Start Timestamp']:
    date_obj = datetime.strptime(date, '%m/%d/%Y %H:%M:%S %p')
    new_date = date_obj.strftime('%Y-%m-%dT%H:%M:%S')
    newdatestart_list.append(new_date)

df['Trip Start Timestamp'] = newdatestart_list
df.head()


Trip ID  \
0  bc6f08423e9f194cb63a0d5ec6c6a2dea001e914   
1  b2e0ede2b0d907551beda4e8b4b83c143010f63d   
2  c2155aedc41a27673cd498d0fea754a18132d7f9   
3  c1e860354f05150021e1bdeb7696314ca3e15608   
4  bdff93a935333338e6f2f04f778ad7713a36c9ad   

                                             Taxi ID Trip Start Timestamp  \
0  07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...  2013-11-16T01:45:00   
1  74605d6aa0c8ba08190a5824f79ef27172c59671b00582...  2016-09-28T04:45:00   
2  4243ab34318764eb5ca51054af503c610b50579594cef6...  2013-12-10T07:15:00   
3  65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...  2014-02-27T09:00:00   
4  8ca267f2246008a052c14ce8c81001149d86916ab274d3...  2016-10-31T04:15:00   

       Trip End Timestamp  Trip Seconds  Trip Miles  Pickup Census Tract  \
0  11/16/2013 01:45:00 AM         480.0        0.15         1.703108e+10   
1  09/28/2016 04:45:00 PM         420.0        0.00         1.703184e+10   
2  12/10/2013 07:30:00 AM         720.0        1.80         1.703128e+10   
3  02/27/2014 09:45:00 AM        2700.0        0.00         1.703108e+10   
4  10/31/2016 04:15:00 PM         660.0        3.00                  NaN   

   Dropoff Census Tract  Pickup Community Area  Dropoff Community Area  \
0          1.703108e+10                    8.0                     8.0   
1          1.703132e+10                   32.0                    32.0   
2          1.703108e+10                   28.0                     8.0   
3          1.703198e+10                    8.0                    76.0   
4                   NaN                   32.0                    35.0   

        ...        Trip Total  Payment Type  \
0       ...              8.05          Cash   
1       ...              8.75   Credit Card   
2       ...              8.44          Cash   
3       ...             50.70   Credit Card   
4       ...             13.50   Credit Card   

                             Company  Pickup Centroid Latitude  \
0                                NaN                 41.907413   
1  Blue Ribbon Taxi Association Inc.                 41.880994   
2                                NaN                 41.879255   
3            Choice Taxi Association                 41.902788   
4                                NaN                 41.878866   

   Pickup Centroid Longitude              Pickup Centroid Location  \
0                 -87.640902  POINT (-87.6409015248 41.9074128162)   
1                 -87.632746  POINT (-87.6327464887 41.8809944707)   
2                 -87.642649   POINT (-87.642648998 41.8792550844)   
3                 -87.626146  POINT (-87.6261455896 41.9027880476)   
4                 -87.625192  POINT (-87.6251921424 41.8788655841)   

  Dropoff Centroid Latitude  Dropoff Centroid Longitude  \
0                 41.892042                  -87.631864   
1                 41.884987                  -87.620993   
2                 41.895033                  -87.619711   
3                 41.979071                  -87.903040   
4                 41.835118                  -87.618678   

              Dropoff Centroid Location Community Areas  
0  POINT (-87.6318639497 41.8920421365)            37.0  
1  POINT (-87.6209929134 41.8849871918)            38.0  
2  POINT (-87.6197106717 41.8950334495)            29.0  
3  POINT (-87.9030396611 41.9790708201)            37.0  
4  POINT (-87.6186777673 41.8351179863)            38.0  

[5 rows x 24 columns]

In [86]:
# Save config information
url = "https://api.darksky.net/forecast/"
exception_count = 0

weather_df=pd.DataFrame()

# calculate begining of itteration from last line in csv file if existant
try:
    with open("Output/weather.csv", "r") as f:
        a = int(pd.read_csv("Output/weather.csv", header=None).tail(1)[0].values[0]+1)
except:
    a = 0

n = a
        
# Build query URL and DataFrame
for i in range(a, len(df)):
    #get info from taxi ride file
    lat = df['Pickup Centroid Latitude'][i]
    lng = df['Pickup Centroid Longitude'][i]
    time = df['Trip Start Timestamp'][i]

    #make API calls with this info and save to DataFrame
    try:
        query_url = url + DarkSkyKey + "/" + str(lat) +"," + str(lng) + "," + time 
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()['currently'] 
        weather_df.loc[i,'Trip ID'] = df['Trip ID'][i]
        weather_df.loc[i,'time'] = time
        weather_df.loc[i,'summary'] = weather_json['summary']
        weather_df.loc[i,'precipIntensity'] = weather_json['precipIntensity']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'temperature'] = weather_json['temperature']
        weather_df.loc[i,'apparentTemperature'] = weather_json['apparentTemperature']
        weather_df.loc[i,'windSpeed'] = weather_json['windSpeed']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'icon'] = weather_json['icon']
    
    except:
        if i-n <= 1: exception_count += 1
        else: exception_count = 0
            
        n = i
        
        if exception_count >= 5: 
            print(f"Request no {i} failed, 5 consecutive requests have failed, ending itteration")
            break
        else: 
            print(f"Request no {i} failed, skipping to next line")
weather_df

Request no 2009 failed, skipping to next line
Request no 2010 failed, skipping to next line
Request no 2011 failed, skipping to next line
Request no 2012 failed, skipping to next line
Request no 2013 failed, 5 consecutive requests have failed, ending itteration


Empty DataFrame
Columns: []
Index: []

In [9]:
try: os.mkdir('Output')
except: pass

try:
    with open("Output/weather.csv", "a") as f:
        weather_df.to_csv(f, header=False)
except:
    weather_df.to_csv("Output/weather.csv")
